In [1]:
!pip install requests pandas numpy


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
import pandas as pd

url = "https://api.spacexdata.com/v4/launches/past"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(f"Datos obtenidos: {len(data)} lanzamientos")
else:
    print("Error al obtener los datos")


Datos obtenidos: 187 lanzamientos


In [3]:
from pandas import json_normalize

df = json_normalize(data)
df.head()


,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,...,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-fa...,https://en.wikipedia.org/wiki/DemoSat,NaN
1,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 301, 'altitude': 289, 'reason': 'har...",Successful first stage burn and transition to ...,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,Lk4zQ2wP-Nc,https://www.space.com/3590-spacex-falcon-1-roc...,https://en.wikipedia.org/wiki/DemoSat,NaN
2,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 140, 'altitude': 35, 'reason': 'resi...",Residual stage 1 thrust led to collision betwe...,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=v0w9p3U8860,v0w9p3U8860,http://www.spacex.com/news/2013/02/11/falcon-1...,https://en.wikipedia.org/wiki/Trailblazer_(sat...,NaN
3,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,True,[],Ratsat was carried to orbit on the first succe...,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=dLQ2tZEH6G0,dLQ2tZEH6G0,https://en.wikipedia.org/wiki/Ratsat,https://en.wikipedia.org/wiki/Ratsat,NaN
4,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,True,[],None,[],[],...,None,None,[],[],http://www.spacex.com/press/2012/12/19/spacexs...,https://www.youtube.com/watch?v=yTaIDooc8Og,yTaIDooc8Og,http://www.spacex.com/news/2013/02/12/falcon-1...,https://en.wikipedia.org/wiki/RazakSAT,NaN


In [4]:
falcon9_id = "5e9d0d95eda69973a809d1ec"
df_falcon9 = df[df['rocket'] == falcon9_id].copy()


In [5]:
def get_total_payload_mass(payloads_list):
    total_mass = 0
    for payload in payloads_list:
        if 'mass_kg' in payload and payload['mass_kg'] is not None:
            total_mass += payload['mass_kg']
    return total_mass if total_mass > 0 else None

df_falcon9['payload_mass_kg'] = df_falcon9['payloads'].apply(get_total_payload_mass)

def get_landing_success(cores_list):
    for core in cores_list:
        if 'landing_success' in core and core['landing_success'] is not None:
            return 1 if core['landing_success'] else 0
    return None

df_falcon9['landing_success'] = df_falcon9['cores'].apply(get_landing_success)


In [6]:
mean_payload = df_falcon9['payload_mass_kg'].mean()
df_falcon9['payload_mass_kg'].fillna(mean_payload, inplace=True)
df_falcon9.dropna(subset=['landing_success'], inplace=True)


C:\Users\Yerko\AppData\Local\Temp\ipykernel_15172\4218615737.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_falcon9['payload_mass_kg'].fillna(mean_payload, inplace=True)
C:\Users\Yerko\AppData\Local\Temp\ipykernel_15172\4218615737.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_falcon9['payload_m

In [7]:
columns_to_keep = ['flight_number', 'date_utc', 'rocket', 'payload_mass_kg', 'launchpad', 'landing_success']
df_clean = df_falcon9[columns_to_keep].copy()
df_clean.to_csv('spacex_launches_clean.csv', index=False)
